In [ ]:
import pandas as pd
import numpy as np
from random import randrange
from math import log2, sqrt,ceil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
bg = np.random.PCG64()

In [ ]:
def gini_index_numerical(above_split, below_split, attribute):
    #UPDATING NOW INPUTS ARE DATAFRAMES
    total_above = len(above_split)
    total_below = len(below_split)
    total = total_above + total_below
    #for each class calculate the individual index
    above_index = 0
    below_index = 0
    #probability point is above
    #TODO OPTIMIZE
    labels = ['0','1']
    above_count = above_split['class'].value_counts()
    below_count = below_split['class'].value_counts()
    for i in range(0,len(labels)):
        #check for empty frames
        try:
            p_above = 0 if above_split.empty else above_count[i] / total_above
        except KeyError:
            p_above = 0
        try:
            p_below = 0 if below_split.empty else below_count[i] / total_below
        except KeyError:
            p_below = 0
        above_index += (p_above * p_above)
        below_index += (p_below * p_below)
    above_index = 1 - above_index
    below_index = 1 - below_index
    return (above_index * (total_above / total)) + (below_index * (total_below / total))

gini_index_numerical takes in two n-d numpy arrays where n represents the number of potential classes. The above_split array represents all of the points located numerically above the split point and the below_split array represents the points with a lesser value.

In [ ]:
def entropy_calculation(splits, labels, total_in_set):
    #calculate the total entropy by calculating the probabilities
    entropy_before_split = 0
    for label in range(0, len(labels)):
        entropy_before_split += labels[label] / total_in_set
    entropy = 0
    for split in splits:
        #for each split calculate the entropy for the given label set
        split_entropy = 0
        total_in_split = len(split)
        label_count = split['class'].value_counts()
        for label in range(0,len(labels)):
            #calculating entropy
            #need to get the values for each label within the split
            #i.e. if split A has 15 values, 4 are class 0: 5 are class 1 and 6 are class 2
            #class 0 would have 4/15 * log2(4/15)
            try:
                label_dist = 0 if split.empty else label_count[label] / total_in_split
            except KeyError:
                label_dist = 0
            split_entropy = split_entropy if label_dist == 0 else (split_entropy + (label_dist * log2(label_dist)))
            #split_entropy += (label_dist * log2(label_dist))
        entropy += split_entropy
    entropy = (-1)*(entropy)
    return entropy
        

In [ ]:
def gini_index_multiclass(splits, labels, total_in_set):
    #calculate for each potential class
    if not splits:
        return 2
    g_index = 0
    for split in splits:
        split_index = 0
        label_count = split['class'].value_counts()
        total_in_split = len(split)
        #for label in labels:
        for label in range(0,len(labels)):
            try:
                p_split = 0 if split.empty else (label_count[label] / total_in_split)
            except (KeyError, IndexError):
                p_split = 0
            split_index += (p_split * p_split)
        split_index = 1-split_index
        g_index += (split_index * (total_in_split / total_in_set))
    return g_index    

The gini_index_multiclass function takes in the splits, the attribute the set was split on, and the potential labels. The potential features should be defined as [0,1,...n-1] where n is the total number of features within the set. Possible feature values could depend on dataset (is pixel on or off, position of pixel, 

In [ ]:
class Node:
    def __init__(self, splitting_point, split_values, split_feature):
        self.split_values = split_values
        self.split_feature = split_feature
        #self.splitting_attribute = splitting_attribute
        self.splitting_point = splitting_point
        #TODO change to k for k classes?
        if split_values == 0:
            #self.children = np.zeros(2)
            self.children = [0 for i in range(0,2)]
        else:
            #self.children = np.zeros(len(split_values))
            self.children = [0 for i in range(0,len(split_values))]
        self.children_split_length = np.zeros(len(self.children))
    
    def new_child_node(self, child_index, child, split_length):
        #print(child_index, child)
        self.children[child_index] = child
        self.children_split_length[child_index] = split_length
        
    def __str__(self):
        a = f"splitting_point {self.splitting_point} , split_feature {self.split_feature}\n"
        for idx, child in enumerate(self.children):
            a += f"child {idx}: {child}"
        return a
    
    def predict(self, point):
        val = point[self.split_feature]
        #child_index = self.children.index(val)
        
        #print(val, self.splitting_point, "retrurning -> ")
        if self.split_values == 0:
            if val >= self.splitting_point:
                #print(self.children[0])
                return self.children[0]
            else:
                #print(self.children[1])
                return self.children[1]
        else:
            #UPDATE THIS FUNCTION HOW TO FOLLOW PATH WHEN NON NUMERICAL?            
            splitting_val = point[self.split_feature].split()
            #GET INDEX OF THE VALUE IN SPLIT_VALUES WHIHC MATCHES INCOMING POINT?
            #'?' represents an unknown value, how to deal with?
            #perhaps follow split with most number of attributes?
            if splitting_val[0] == "?":
                #unknown follow feature attribute with most values?
                #return index of max split value
                child_index = np.argmax(self.children_split_length)
            else:
                child_index = self.split_values.index(splitting_val[0])
            return self.children[child_index]

The Node class represents each node within the tree. The splitting attribute and point determines the axis and value the node splits on. The above and below values represent the next node or label based on the splitting point.

In [ ]:
def split_continuous(value, attribute, examples):
    #split where item in row > or < value
    above_values = examples.loc[examples[attribute] >= value]
    below_values = examples.loc[examples[attribute] < value]
    return [above_values, below_values]
    

In [ ]:
def split_discrete(feature_name, feature_values, examples):
    #split dataset based on all possible values
    #TODO make this dynamically adjustable for when |features| > 2!
    splits = []
    for value in feature_values:
        j_value = value.rjust(len(value)+1)
        subset = examples.loc[examples[feature_name] == j_value]
        splits.append(subset)
    return splits                                   

In [ ]:
def find_best_split(examples, features, calculate_purity):
    #given a dataframe -> examples find the best splitting value within based
    #on the given purity calculation
    #for row in examples
        #for attribute in row
            #value = row[attribute]
            #split = split(value, examples)
            #purity = calculate_purity(split)
            #if purity better than best purity is best
    #TODO OPTIMIZE THIS
    best_purity = 2
    best_node = None
    best_splits = []
    for index, row in examples.iterrows():
        #loop through all features determining which are continous and which are discrete
        #discrete -> pixel on / off or values such as couldy/rainy/windy
        for feature_name, values in features.items():
            
            if(row[feature_name] == "?"):
                print("UNKNOWN")
            if(values == 0):
                #continous split based on current value in the row
                try:
                    splits = split_continuous(row[feature_name],feature_name,examples)
                except (IndexError,KeyError):
                    splits = []
            else:
                #split based on 
                splits = split_discrete(feature_name, values, examples)           
            #split dataframe into 2 dataframes
            #splits = split(row[val], val, examples)
            #purity = calculate_purity(above_split, below_split, val)
            purity = calculate_purity(splits, examples['class'].value_counts(), len(examples))
            if purity < best_purity:
                best_purity = purity
                #create new node based on best attribute and value
                best_node = Node(row[feature_name], values, feature_name)
                best_splits = splits
    return best_node, best_splits
        

In [ ]:
def most_common_class(examples):
    classes = examples['class'].value_counts()
    return classes.idxmax()

In [ ]:
def number_of_classes_in_samples(examples):
        return len(examples['class'].value_counts())

In [ ]:
def learn_tree(examples, features, parent_examples, depth, max_depth, calculate_purity):
    #work with ending conditions, if no more examples left then return most common class in parent examples
    if examples.empty or (number_of_classes_in_samples == 0):
        return most_common_class(parent_examples)
    elif (number_of_classes_in_samples(examples) == 1) or (depth >= max_depth):
        return most_common_class(examples)
    else:
        #find the best split possible within the current split points
        #splits[] = [above, below]
        root_node, splits = find_best_split(examples, features, calculate_purity)
        for i in range(0,len(splits)):
            #above and below calculations
            #split should consist of a subset of examples
            new_node = learn_tree(splits[i],features, examples,depth-1,max_depth, calculate_purity)
            root_node.new_child_node(i,new_node, len(splits[i]))
        return root_node

In [ ]:
def tree_prediction(point, root_node):
    prediction = root_node.predict(point)
    while isinstance(prediction, Node):
        prediction = prediction.predict(point)
    return prediction

In [ ]:
def get_subset(data_frame, subset_length):
    #randomly sample rows from the dataframe up to length n
    #where n is the specified subset length
    subset = data_frame.sample(n=subset_length, random_state=bg)    
    return subset

In [ ]:
def random_forest(data_frame, features, subset_length, max_depth, num_of_trees, purity_calculation):
    trees = []
    for i in range(0, num_of_trees):
        #generate a random subsample
        subset = get_subset(data_frame, subset_length)
        #create the tree and append to list
        tree = learn_tree(subset, features, None, 0, max_depth, purity_calculation)
        trees.append(tree)
    return trees

In [92]:
def predict_data_with_trees(testing_data, trees, num_of_trees):
    predicted_labels = []
    for index, row in testing_data.iterrows():
        predictions = np.zeros(num_of_trees)
        for i in range(0,num_of_trees):
            #casting to int because bincount does not like floating point
            #print(f"looking at tree {trees[i]}")
            #print(f"NEW ELEMENT: {row}")
            predictions[i] = tree_prediction(row, trees[i])
            #print(f"result {predictions[i]}")
        #select maximum class, voting method
        #print("done with predictions")
        most_voted_class = np.bincount(predictions.astype(np.int64)).argmax()
        predicted_labels.append(most_voted_class)
    return predicted_labels
        
        

In [ ]:
test_data = {'id': [1,2,3,4,5,6,7,8,9,10], '0': [1,12,4.3,6,14,2,9,14,5,4],'1':[4.5,7,1,3,4.8,15,17,7,1.9,10],'class':[0,1,0,0,1,0,1,1,0,0]}

In [ ]:
df = pd.DataFrame(data=test_data)

In [ ]:
max_depth = 4

In [ ]:
testing = {'id': [1,2,3], '0': [1,19,11],'1':[4,12,11],'class':[0,1,1]}
tdf = pd.DataFrame(data=testing)
subset_length = sqrt(len(testing))
predictions = []
for num_of_trees in tree_nums:
    trained_trees = random_forest(df,subset_length,max_depth,num_of_trees,entropy_calculation)
    for root_node in trained_trees:
        print(root_node)
    predictions = predict_data_with_trees(tdf,trained_trees,num_of_trees)
print(predictions)

In [ ]:
features = {"sepal length":0,"sepal width":0,"petal length":0,"pedal width":0}
f_names = ["sepal length","sepal width","petal length","petal width","class"]
newdf = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",names=f_names)
labelEnc = LabelEncoder()
labelEnc.fit(newdf['class'])
newdf['class'] = labelEnc.transform(newdf['class'])
newdf.tail()

In [ ]:
features["sepal length"]

In [ ]:
labels = newdf['class'].value_counts()
labels
tree_nums = [10, 50, 150]
max_depth = 10

length_of_data = len(newdf)
train_ratio = 0.7
length_of_train = length_of_data * train_ratio
train, test = train_test_split(newdf, test_size = 0.3, random_state = 43, shuffle=True)
subset_length = ceil(sqrt(len(train)))

In [ ]:
iris_predictions = []
for num_of_trees in tree_nums:
    trained_trees = random_forest(train, features, subset_length, max_depth, num_of_trees, gini_index_multiclass)
    predictions = predict_data_with_trees(test, trained_trees, num_of_trees)
    i = 0
    correct = 0
    total = 0
    for ind, val in test['class'].items():
        if predictions[i] == val:
            correct+=1
        i+=1
        total+=1
    print(f"Correct classifications: {correct} | total {total}")
        
        
    
    
    

In [ ]:
f_names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','class']
adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names = f_names)
adult.head()

In [ ]:
features = {'age':0,
            'workclass':["Private", "Self-emp-not-inc", "Self-emp-inc", "Federal-gov", "Local-gov", "State-gov", "Without-pay", "Never-worked"],
            'fnlwgt':0,
            'education':['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'],
            'education-num':0,
            'marital-status':['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'],
            'occupation':['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'],
            'relationship':['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'],
            'race':['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'],
            'sex':['Female', 'Male'],
            'capital-gain':0,
            'capital-loss':0,
            'hours-per-week':0,
            'native-country':['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 
                                'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']}
labelEnc.fit(adult['class'])
adult['class'] = labelEnc.transform(adult['class'])
adult.head()

In [93]:
adult_predictions = []
max_depth = 10
length_of_data = len(adult)
train_ratio = 0.7
train, test = train_test_split(adult, test_size = 0.3, random_state = 42, shuffle=True)
subset_length = ceil(sqrt(len(train)))
print(subset_length)
tree_nums = [5,10,15]
for num_of_trees in tree_nums:
    trained_trees = random_forest(train, features, subset_length, max_depth, num_of_trees, gini_index_multiclass)
    predictions = predict_data_with_trees(test, trained_trees, num_of_trees)
    i = 0
    correct = 0
    total = 0
    for ind, val in test['class'].items():
        if predictions[i] == val:
            correct+=1
        i+=1
        total+=1
    print(f"Correct classifications: {correct} | total {total}")
        
        

151
Correct classifications: 7804 | total 9769
Correct classifications: 7940 | total 9769
Correct classifications: 8035 | total 9769


In [ ]:
len(trained_trees)

In [ ]:
pred = predict_data_with_trees(test, trained_trees, num_of_trees)